In [9]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
from pyspark.sql.functions import *
import pandas as pd
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

In [10]:

merchant_fraud = spark.read.option("header",True).csv('../data/tables/merchant_fraud_probability.csv')
tbl_merchants = spark.read.parquet('../data/tables/tbl_merchants.parquet')


In [11]:
merchant_fraud = merchant_fraud.select(col("merchant_abn"),col("fraud_probability"),to_date(col("order_datetime"),"yyyy-MM-dd").alias("date"))

print(merchant_fraud.count())
merchant_fraud.filter("date > '2021-02-28'")
merchant_fraud.filter("date < '2022-08-28'")
print(merchant_fraud.count())

114
114


In [12]:
merchant_fraud_group = merchant_fraud.groupBy('merchant_abn').agg(F.count('fraud_probability').alias('fraud_count_abn'))
merchant_fraud_group.limit(2)

merchant_abn,fraud_count_abn
67264251405,1
43083074133,1


In [13]:
tbl_merchants_pd = tbl_merchants.toPandas()
for i in range(int(tbl_merchants_pd['tags'].count())):
    tbl_merchants_pd['tags'].iloc[i] = tbl_merchants_pd['tags'].iloc[i].replace(r'[', r'(').replace(r']', r')')

In [14]:
merchant_tags = tbl_merchants_pd['tags'].str.split(')', expand=True)

for row in range(int(len(merchant_tags))):
    for col in range(3):
        merchant_tags.iloc[row,col] = merchant_tags.iloc[row,col].replace(r'((', r'').replace(r', (', r'').replace(r'take_rate:', r'')
merchant_tags.rename(columns = {0 : 'store_type', 1 : 'revenue_levels', 2 : 'take_rate'}, inplace = True)
merchant_tags = merchant_tags[['store_type', 'revenue_levels', 'take_rate']]

In [15]:
tbl_merchants_pd[['store_type', 'revenue_levels', 'take_rate']] = merchant_tags[['store_type', 'revenue_levels', 'take_rate']]
#merchant.drop(columns=['tags'])
tbl_merchants_pd['store_type'] = tbl_merchants_pd['store_type'].str.lower()

for i in range(len(tbl_merchants_pd)):
    tbl_merchants_pd['store_type'][i] = ' '.join(tbl_merchants_pd['store_type'][i].split())
 

/Users/seren/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
merchants = tbl_merchants_pd.drop(columns = ['name','tags'])
merchants.head()
#merchants.to_parquet(f"../data/curated/final_merchants.parquet")

,merchant_abn,store_type,revenue_levels,take_rate
0,10023283211,"furniture, home furnishings and equipment shop...",e,take rate: 0.18
1,10142254217,"cable, satellite, and other pay television and...",b,take rate: 4.22
2,10165489824,"jewelry, watch, clock, and silverware shops",b,take rate: 4.40
3,10187291046,"watch, clock, and jewelry repair shops",b,take rate: 3.29
4,10192359162,"music shops - musical instruments, pianos, and...",a,take rate: 6.33
